In [1]:
from Classes.KNumNeighborsClassifier import KNumNeighborsClassifier
from Classes.KNumNeighborsTeacher import KNumNeighborsTeacher
from Classes.NaiveBayesianClassificator import NaiveBayesianClassificator
from Classes.NaiveBayesianTeacher import NaiveBayesianTeacher
from Classes.BaggingContainer import BaggingContainer
from Classes.BaggingTeacher import BaggingTeacher

from Functions.distance_functions import euclid_distance
from Functions.functions_of_priority import max_count_class
from Functions.spreading_functions import normal_spread
from Functions.split_functions import bootstrap_split_of_train_data
from Functions.decision_functions import mean_value
from Functions.metrics import *

from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from random import getrandbits

In [2]:
def random_split_on_two(x, n):
    return [np.array(list(map(lambda a: getrandbits(1), range(x.shape[0]))), dtype=bool) for i in range(n)]


In [3]:
models = pd.read_csv('../Data/DataLaba2/neo_task_smoted.csv')
X_train, X_test, y_train, y_test = train_test_split(models.drop('hazardous', axis=1), models['hazardous'], test_size=0.1, random_state=42)

In [4]:
knn = KNumNeighborsClassifier(k=5, 
                            distance=euclid_distance, 
                            function_of_priority=max_count_class)


In [5]:
nbc = NaiveBayesianClassificator(lambda a: a > 0)


In [6]:
bagging_container = BaggingContainer(predictors=[nbc, nbc, nbc],
                                     out_function=mean_value)
bagging_teacher = BaggingTeacher(begging_container=bagging_container, 
                                 teachers=[NaiveBayesianTeacher,
                                           NaiveBayesianTeacher,
                                           NaiveBayesianTeacher],
                                 teachers_params=[[], []],
                                 split_function=bootstrap_split_of_train_data,
                                 )
bagging_teacher.teach(X_train.to_numpy()[0:100], y_train.to_numpy()[0:100], [
                      normal_spread, normal_spread, normal_spread, normal_spread])


In [7]:
y_predicted = bagging_container.predict(X_test.to_numpy())
y_predicted


0.6110365853658537
0.6110365853658537


array([1, 0, 1, ..., 1, 1, 1], dtype=int32)

In [8]:
print(
    f'TP : {TruePositive(y_predicted, y_test)}',
    f'TN : {TrueNegative(y_predicted, y_test)}',
    f'FN : {FalseNegative(y_predicted, y_test)}',
    f'FP : {FalsePositive(y_predicted, y_test)}',
    f'Accuracy : {accuracy(y_predicted, y_test)}',
    f'Specificity : {specificity(y_predicted, y_test)}',
    f'Prcision : {precision(y_predicted, y_test)}',
    f'Recall : {recall(y_predicted, y_test)}',
    f'F1 : {f1(y_predicted, y_test)}',
    sep='\n'
    )

TP : 7882
TN : 6008
FN : 371
FP : 2139
Accuracy : 0.8469512195121951
Specificity : 0.737449367865472
Prcision : 0.7865482486777766
Recall : 0.9550466497031382
F1 : 0.8626463828390062
